# Data Collection and Preprocessing

In this notebook, we use the Green Metric Tool API to collect and preprocess the power and traffic data captured during simulated email user sessions.
This involves checking no errors occured during the tests.

### Importing libraries

In [4]:
import pandas as pd # For data manipulation
import numpy as np # For data manipulation
import requests # For HTTP Requests to the API
import seaborn as sns # For charts
from IPython.display import display # To dataframe displaying
import scipy.stats as stats # For stats
import pingouin as pg # For stats
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
from datetime import datetime, timezone
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

#Set display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect the width
pd.set_option('display.max_colwidth', None)  # Show full content of each column

- Define the URL and other constants to query the GMT API

In [5]:
URL = 'http://api.green-coding.internal:9142'
REPO = '/home/jason/2024-loco-security-sustainability-artifact/simulator'
RUNS = '/v1/runs'
NOTES = '/v1/notes/'
MEASUREMENTS = '/v1/measurements/single/'

- Define functions to:
    - get all the runs infos (test_name, run_ids, time)
    - get all run ids of tests which names include a substring
    - get the test name of a given run id
    - get the notes from a given run id
    - print the logs of a given run id
    - get the measurment of a given run id

In [6]:
# Function to get all runs
def get_runs():
    try:
        # Fetch data from the API
        response = requests.get(URL + RUNS)
        response.raise_for_status()  # Check if the request was successful
        
        data = response.json()
        
        # Convert the relevant part of the data into a DataFrame
        runs = pd.DataFrame.from_dict(data['data'])
        runs = runs[[0, 1, 4]]
        runs.columns = ['id', 'name', 'time']
        # Convert the 'time' column (containing UTC timestamp strings) to UTC timestamps
        runs['time'] = pd.to_datetime(runs['time'])
        
        return runs
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as e:
        print(f"Failed to decode JSON: {e}")

# Function to get all the run IDs for a test name containing a given substring
def get_ids(substring="", time=False):
    runs = get_runs()
    filtered_runs = runs[runs['name'].str.contains(substring, na=False)]
    if time:
        return filtered_runs['id'].tolist(), filtered_runs['time'].tolist()
    return filtered_runs['id'].tolist()

# Function to get the test name of a run given its ID
def get_name(id):
    runs = get_runs()
    run = runs[runs['id'] == id]
    
    if not run.empty:
        return run.iloc[0]['name']
    else:
        return None

# Function to create the notes (i.e., logs) DataFrame from an ID
def get_notes(id):
    try:
        response = requests.get(URL + NOTES + id)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()
        notes = pd.DataFrame.from_dict(data['data'])
        
        # Set the appropriate column names
        notes.columns = ['id', 'container', 'log', 'time']

        # Sort the logs by time
        notes.sort_values(by='time', ascending=False, inplace=True)
        
        return notes
        
    except Exception as e:
        print(f"Failed to fetch or process notes for ID {id}: {e}")
        # Return an empty DataFrame with the expected column names
        return pd.DataFrame(columns=['id', 'container', 'log', 'time'])

# Function to print the logs for a given ID
def print_logs(id):
    # print the notes (logs) for the given ID
    notes_df = get_notes(id)
    
    if notes_df.empty:
        print(f"No logs found for ID {id}.")
        return
    
    # Print the logs in order
    for index, row in notes_df.iterrows():
        print(f"Time: {row['time']}, Log: {row['log']}")

# Function to get the measurement DataFrame from an ID
def get_measurements(id):
    try:
        response = requests.get(URL + MEASUREMENTS + id)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()
        measurement = pd.DataFrame.from_dict(data['data'])
        
        # Set the appropriate column names
        measurement.columns = ['detail_name', 'time', 'metric', 'value', 'unit']
        return measurement

    except Exception as e:
        print(f"Failed to fetch measurements for ID {id}: {e}")
        # Return an empty DataFrame in case of an error
        return pd.DataFrame(columns=['detail_name', 'time', 'metric', 'value', 'unit'])

- We use a DataFrame to map each (provider, adblock condition, pgp condition, functional unit) combination to the corresponding test name and the action to monitor within those tests.

    EXAMPLE:
    - Considering the 'reply' functional unit for the 'gmail' provider with adblock enabled, we look up the entry `['gmail', 'enabled', 'disabled', 'reply']`. This entry directs us to the test name `'2025_gmail_session_adblock.json'` and specifies the action to monitor: `'reply'`.

    - Considering the 'attachment' functional unit (i.e., send with attachment file) for the 'mysolution' provider with adblock disabled and PGP enabled, we look up the entry `['mysolution', 'disabled', 'enabled', 'attachment']`. This entry directs us to the test name `'2025_mysolution_session_nopgp.json'` and specifies the action to monitor: `'send_mail_5MB'`.


In [7]:
#PROVIDERS = ['gmail', 'outlook', 'proton', 'mysolution', 'mysolution_remote']
PROVIDERS = ['mysolution_remote']
ADBLOCK_CONDITION = ['enabled', 'disabled']
PGP_CONDITIONS = ['enabled', 'disabled']
FUNCTIONAL_UNITS = ['login', 'logout', 'noattachment', 'attachment', 'read', 'reply', 'delete', 'session']

    
def get_test_name_and_action(provider, adblock, pgp, functional_unit):

    test_name = "2025_"
    action = None
    
    if provider == 'mysolution_remote':
        test_name += 'mysolution_session_'
        latency = 'remote50'
    elif provider == 'mysolution_on_site':
        test_name += 'mysolution_session_'
        latency = 'onsite'
    else:
        test_name += f"{provider}_session_"
        latency = False

    if functional_unit == 'login':
        action = 'login'
    elif functional_unit == 'logout':
        action = 'logout'
    elif functional_unit == 'attachment':
        action = 'send_mail_5MB'
    elif functional_unit == 'noattachment':
        action = 'send_mail_0MB'
    elif functional_unit == 'read':
        action = 'read_first'
    elif functional_unit == 'reply':
        action = 'reply'
    elif functional_unit == 'delete':
        action = 'delete_first'
    elif functional_unit == 'session':
        action = 'session'

    test_name += 'adblock' if adblock == 'enabled' else 'noadblock'

    if 'mysolution' in provider:
        test_name += '_pgp' if pgp == 'enabled' else '_nopgp'

    if latency:
        test_name += f'_{latency}'
    test_name += '.json'

    return test_name, action

    
# Generate all combinations of provider, adblock, pgp, and functional_unit
tests_and_actions = [
    [provider, adblock, pgp, functional_unit] + list(get_test_name_and_action(provider, adblock, pgp, functional_unit))
    for provider in PROVIDERS
    for adblock in ADBLOCK_CONDITION
    for pgp in PGP_CONDITIONS
    for functional_unit in FUNCTIONAL_UNITS
    if not (pgp == 'enabled' and not 'mysolution' in provider)  # Exclude 'enabled' PGP for all providers except 'mysolution'
    if not ('mysolution' in provider and adblock == 'enabled') # Exclude 'enabled' ADBLOCK for provider 'mysolution'
]

# Create the DataFrame
TESTS_AND_ACTIONS = (pd.DataFrame(tests_and_actions, columns=['PROVIDER', 'ADBLOCK_CONDITION', 'PGP_CONDITION', 'FUNCTIONAL_UNIT', 'TEST', 'ACTION']) \
    .set_index(['PROVIDER', 'ADBLOCK_CONDITION', 'PGP_CONDITION', 'FUNCTIONAL_UNIT']))

TESTS_AND_ACTIONS

TEST  \
PROVIDER          ADBLOCK_CONDITION PGP_CONDITION FUNCTIONAL_UNIT                                                          
mysolution_remote disabled          enabled       login              2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  logout             2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  noattachment       2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  attachment         2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  read               2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  reply              2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  delete             2025_mysolution_session_noadblock_pgp_remote50.json   
                                                  session            2025_mysolution_session_noadblock_pgp_remote50.json   
                                    disabled      login            2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  logout           2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  noattachment     2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  attachment       2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  read             2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  reply            2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  delete           2025_mysolution_session_noadblock_nopgp_remote50.json   
                                                  session          2025_mysolution_session_noadblock_nopgp_remote50.json   

                                                                          ACTION  
PROVIDER          ADBLOCK_CONDITION PGP_CONDITION FUNCTIONAL_UNIT                 
mysolution_remote disabled          enabled       login                    login  
                                                  logout                  logout  
                                                  noattachment     send_mail_0MB  
                                                  attachment       send_mail_5MB  
                                                  read                read_first  
                                                  reply                    reply  
                                                  delete            delete_first  
                                                  session                session  
                                    disabled      login                    login  
                                                  logout                  logout  
                                                  noattachment     send_mail_0MB  
                                                  attachment       send_mail_5MB  
                                                  read                read_first  
                                                  reply                    reply  
                                                  delete            delete_first  
                                                  session                session

- Metadata Collection and Sample Counting :

    We collect metadata related to energy and traffic measurements for different **providers** and **functional units** with/without **ad-blocker** and with/without **PGP**.

    - **`collect_metadata`**: Gathers metadata for a specific provider and functional unit, including start and end timestamps, ensuring no errors occur during the functional unit execution to validate the sample.
    - **`collect_all_metadata`**: Aggregates metadata for all providers and functional units.

    Additionally, we monitor the number of valid samples collected:

    - **`count_distinct_samples`**: Counts valid and missing samples based on unique start and end timestamps.


In [8]:

def collect_metadata(provider, adblock, pgp, functional_unit, test_name, action, from_date=None, till_date=None):
    """
    Collect metadata required to fetch energy measurements.
    This includes gathering the run ID and the time period when a functional unit was performed.
    A period is considered valid if no error occurred during the functional unit execution.

    Args:
        provider (str): The provider for which metadata is being collected.
        adblock (str): The adblock condition: enabled/disabled.
        pgp (str): The pgp condition: enabled/disabled.
        functional_unit (str): The functional unit to monitor.
        test_name (str): The name of the test which perform the functional units for the provider under given adblock/pgp conditions.
        action (str): The action that triggers the start/end logs.
        from_date (str, optional): Start date in 'dd-mm-yyyy' format. If None, no start limit is applied.
        till_date (str, optional): End date in 'dd-mm-yyyy' format. If None, no end limit is applied.

    Returns:
        pd.DataFrame: A DataFrame containing metadata with columns:
                      ['TEST', 'RUN_ID', 'PROVIDER', 'ADBLOCK_CONDITION', 'PGP_CONDITION', 
                       'FUNCTIONAL_UNIT', 'START_TS', 'END_TS'].
    """
    
    columns = ['TEST', 'RUN_ID', 'PROVIDER', 'ADBLOCK_CONDITION', 'PGP_CONDITION', 'FUNCTIONAL_UNIT', 'START_TS', 'END_TS']
    metadata_records = []  # List to store metadata records
    ids, timestamps = get_ids(test_name, time=True)  # Fetch the test run IDs and their corresponding timestamps

    for run_id, run_timestamp in zip(ids, timestamps, strict=True):

        # Check if the timestamp fits within the date range (if provided)
        if from_date and run_timestamp < pd.to_datetime(from_date, format="%d-%m-%Y", utc=True):
            continue  # Skip records that are before the 'from' time
        if till_date and run_timestamp > pd.to_datetime(till_date, format="%d-%m-%Y", utc=True):
            continue  # Skip records that are after the 'till' time

        logs = get_notes(run_id)  # Get logs for the run
        end_log_time = None  # Variable to track the latest "END" log timestamp

        for _, row in logs.iterrows():
            log_message = row['log']
            timestamp = row['time']

            # Detect end log for the specific action
            if f"END: {action}" in log_message:
                end_log_time = timestamp
                continue

            # Invalidate end time if there's an error during action execution
            if 'ERROR' in log_message or 'stacktrace' in log_message:
                end_log_time = None
                continue

            # Collect metadata once a valid start-end pair is found
            if f"START: {action}" in log_message and end_log_time is not None:
                metadata_records.append((test_name, run_id, provider, adblock, pgp, functional_unit, timestamp, end_log_time))
                end_log_time = None  # Reset after recording
    
    return pd.DataFrame(metadata_records, columns=columns)



def collect_all_metadata(from_date=None, till_date=None):
    """
    Collect metadata for all providers and functional units.
    """
    metadata_list = [
        collect_metadata(provider, adblock, pgp, functional_unit, row['TEST'], row['ACTION'], from_date, till_date)
        for (provider, adblock, pgp, functional_unit), row in TESTS_AND_ACTIONS.iterrows()
    ]

    return pd.concat(metadata_list, ignore_index=True,)

def count_distinct_samples(metadata: pd.DataFrame):
    """
    Count the number of valid samples for each (provider, functional_unit) combination.
    A valid sample is defined as a unique pair of start and end timestamps for each (TEST, ACTION).

    Args:
        metadata (pd.DataFrame): The metadata DataFrame to count samples from.

    Returns:
        pd.DataFrame: A DataFrame containing the sample counts and missing sample counts.
    """
    return (metadata.groupby(['PROVIDER', 'ADBLOCK_CONDITION', 'PGP_CONDITION', 'FUNCTIONAL_UNIT'])
                .agg(SAMPLE_COUNT=('START_TS', 'count'), 
                    MISSING_SAMPLES=('START_TS', lambda x: max(0, 100 - x.count())))  # Count missing samples
            )

In [9]:
# Collect the full metadata DataFrame
# Tests performed in Switzerland on a ethernet link connection : '18-02-2025 - 25-02-2025'

metadata = collect_all_metadata(from_date='18-02-2025')

# Count distinct test-action pairs for the filtered metadata
count_distinct_samples(metadata)


Failed to fetch or process notes for ID 041e09b7-a91c-46da-98d7-5441816eaa36: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 7ec1373d-78e7-41a8-a3f5-378cdd63b409: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 041e09b7-a91c-46da-98d7-5441816eaa36: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 7ec1373d-78e7-41a8-a3f5-378cdd63b409: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 041e09b7-a91c-46da-98d7-5441816eaa36: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 7ec1373d-78e7-41a8-a3f5-378cdd63b409: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 041e09b7-a91c-46da-98d7-5441816eaa36: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 7ec1373d-78e7-41a8-a3f5-378cdd63b409: Expecting value: line 1 column 1 (char 0)
Failed to fetch or process notes for ID 

SAMPLE_COUNT  \
PROVIDER          ADBLOCK_CONDITION PGP_CONDITION FUNCTIONAL_UNIT                 
mysolution_remote disabled          disabled      attachment                180   
                                                  delete                    158   
                                                  login                     180   
                                                  logout                    180   
                                                  noattachment              180   
                                                  read                      309   
                                                  reply                     146   
                                                  session                   139   
                                    enabled       attachment                100   
                                                  delete                     83   
                                                  login                     100   
                                                  logout                    100   
                                                  noattachment              100   
                                                  read                      169   
                                                  reply                      84   
                                                  session                    82   

                                                                   MISSING_SAMPLES  
PROVIDER          ADBLOCK_CONDITION PGP_CONDITION FUNCTIONAL_UNIT                   
mysolution_remote disabled          disabled      attachment                     0  
                                                  delete                         0  
                                                  login                          0  
                                                  logout                         0  
                                                  noattachment                   0  
                                                  read                           0  
                                                  reply                          0  
                                                  session                        0  
                                    enabled       attachment                     0  
                                                  delete                        17  
                                                  login                          0  
                                                  logout                         0  
                                                  noattachment                   0  
                                                  read                           0  
                                                  reply                         16  
                                                  session                       18

- First we check which metrics we have at our disposal, their name and units

In [ ]:
test_id = get_ids("2025")[0]
print(get_measurements(test_id)[['metric', 'unit']].drop_duplicates())

                                  metric   unit
0          cpu_energy_rapl_msr_component     mJ
27326   cpu_utilization_cgroup_container  Ratio
54369      cpu_utilization_procfs_system  Ratio
81673   memory_energy_rapl_msr_component     mJ
109009     memory_total_cgroup_container  Bytes
136114       network_io_cgroup_container  Bytes
163195     psu_energy_ac_xgboost_machine     mJ


- Then we rename those metrics and define our own computed metrics

In [10]:
METRICS = [
    'CPU_ENERGY',                   # cpu_energy_rapl_msr_component
    'MEMORY_ENERGY',                # memory_energy_rapl_msr_component
    'NETWORK_IO',                   # network_io_cgroup_container
    'ESTIMATED_MACHINE_ENERGY',     # psu_energy_ac_xgboost_machine
    'DURATION']

COMPUTED_METRICS = ['CPU_POWER',
                    'MEMORY_POWER',
                    'ESTIMATED_MACHINE_POWER']
UNITS = {
    'CPU_ENERGY': 'mJ',
    'MEMORY_ENERGY': 'mJ',
    'NETWORK_IO' : 'Byte',
    'ESTIMATED_MACHINE_ENERGY' : 'mJ',
    'DURATION' : 'µs',
    'CPU_POWER' : 'W',
    'MEMORY_POWER' : 'W',
    'ESTIMATED_MACHINE_POWER' : 'W'
}

- Collect and Organise Metric Data into a DataFrame :

    **Gathering Measurement Data**

    In the `get_data` function, we collect measurement data for each sample in the `metadata` DataFrame. We extract key metadata, including test names, run IDs, provider names, adblock conditions, pgp condition, functional units, and the start and end timestamps for each sample. For each time period, we then compute the energy- or data-related metrics—such as memory energy, CPU energy, estimated machine energy, and network I/O—using the `get_metric_stat` function.

    **Storing the Collected Data**

    We store the computed values in a DataFrame. The columns of this DataFrame are: `TEST`, `RUN_ID`, `PROVIDER`, ,`ADBLOCK_CONDITION`,
                                    `PGP_CONDITION`,`FUNCTIONAL_UNIT`, `START_TS`, `DURATION`, `MEMORY_ENERGY`, `CPU_ENERGY`, `ESTIMATED_MACHINE_ENERGY`, and `NETWORK_IO`. Then we export it as a csv file


In [11]:
def get_metric_stat(measurements, metric, start_ts, end_ts, func):

    # Filter to keep records of the desired metric
    measurements = measurements[measurements['metric'] == metric]
    # Filter to keep records in desired period
    measurements = measurements[(measurements['time'] >= start_ts) & (measurements['time'] < end_ts)]
    # Compute the statistic
    stat = func(measurements['value'])

    return stat

def get_data(metadata:pd.DataFrame):

    data_records = []  # Store tuples for DataFrame creation

    for run_id, run_metadata in metadata.groupby('RUN_ID') :

        # Get measurment for the run
        run_measurements = get_measurements(run_id)
        run_measurements.sort_values(by='time', ascending=True, inplace=True)

        for _, row in run_metadata.iterrows():

            # Extract Metadata:
            test = row['TEST']
            provider = row['PROVIDER']
            adblock_condition = row['ADBLOCK_CONDITION'] 
            pgp_condition = row['PGP_CONDITION']
            functional_unit = row['FUNCTIONAL_UNIT']
            start_ts = row['START_TS']
            end_ts = row['END_TS']
            duration = end_ts - start_ts

            # Compute Data
            memory_energy = get_metric_stat(run_measurements, 'memory_energy_rapl_msr_component', start_ts, end_ts, sum)
            cpu_energy = get_metric_stat(run_measurements, 'cpu_energy_rapl_msr_component', start_ts, end_ts, sum)
            estimated_machine_energy = get_metric_stat(run_measurements, 'psu_energy_ac_xgboost_machine', start_ts, end_ts, sum)
            network_io = get_metric_stat(run_measurements, 'network_io_cgroup_container', start_ts, end_ts, lambda x: x.iloc[-1] - x.iloc[0] if len(x) > 1 else 0)

            # Store records
            data_records.append((test,
                                run_id,
                                provider,
                                adblock_condition,
                                pgp_condition,
                                functional_unit,
                                start_ts,
                                duration,
                                memory_energy,
                                cpu_energy,
                                estimated_machine_energy,
                                network_io))
        
    data = pd.DataFrame(  columns = ['TEST',
                                    'RUN_ID',
                                    'PROVIDER',
                                    'ADBLOCK_CONDITION',
                                    'PGP_CONDITION',
                                    'FUNCTIONAL_UNIT',
                                    'START_TS',
                                    'DURATION',
                                    'MEMORY_ENERGY',
                                    'CPU_ENERGY',
                                    'ESTIMATED_MACHINE_ENERGY',
                                    'NETWORK_IO'],
                        data = data_records).sort_values(by=[   'PROVIDER',
                                                                'ADBLOCK_CONDITION',
                                                                'PGP_CONDITION',
                                                                'FUNCTIONAL_UNIT'])
    return data

In [12]:
data = get_data(metadata=metadata)
data.to_csv('2025_data_test_bis.csv', index=False)
data.head()

,TEST,RUN_ID,PROVIDER,ADBLOCK_CONDITION,PGP_CONDITION,FUNCTIONAL_UNIT,START_TS,DURATION,MEMORY_ENERGY,CPU_ENERGY,ESTIMATED_MACHINE_ENERGY,NETWORK_IO
30,2025_mysolution_session_noadblock_nopgp_remote50.json,03141515-4b75-4309-9846-c80ee9e996f8,mysolution_remote,disabled,disabled,attachment,1750029558055899,46954165,53582,231727,1036851,5390495
31,2025_mysolution_session_noadblock_nopgp_remote50.json,03141515-4b75-4309-9846-c80ee9e996f8,mysolution_remote,disabled,disabled,attachment,1750029386692675,44034378,50130,209339,967364,5376158
32,2025_mysolution_session_noadblock_nopgp_remote50.json,03141515-4b75-4309-9846-c80ee9e996f8,mysolution_remote,disabled,disabled,attachment,1750029215716196,42121300,48003,204336,926375,5372101
33,2025_mysolution_session_noadblock_nopgp_remote50.json,03141515-4b75-4309-9846-c80ee9e996f8,mysolution_remote,disabled,disabled,attachment,1750029042826856,42003876,47568,199047,921029,5378477
34,2025_mysolution_session_noadblock_nopgp_remote50.json,03141515-4b75-4309-9846-c80ee9e996f8,mysolution_remote,disabled,disabled,attachment,1750028868455855,44445336,50469,214453,979741,5373368
